<a href="https://colab.research.google.com/github/steliosg23/PDS-A2/blob/main/SUBMISSION%20Finetuned%20PubMedBERT%20PDS%20A2%20Food%20Hazard%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necessary packages and import libraries
This section includes all the necessary imports for data manipulation, model training, and evaluation.
It also imports libraries for handling tokenization, model configuration, and metrics.


In [ ]:
from google.colab import drive
import pandas as pd
import torch
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import os
from shutil import make_archive
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Mount Google Drive


In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load and preview the training dataset
The dataset containing incident reports is loaded from Google Drive.
We remove any unnecessary columns like 'Unnamed: 0'.


In [ ]:
train_path = '/content/drive/MyDrive/Data/incidents_train.csv'
df = pd.read_csv(train_path)
df = df.drop(columns=['Unnamed: 0'])


# Define a function to clean text data
This function removes special characters, converts text to lowercase, and strips extra whitespace.
It is essential to clean the text data for better model performance.


In [ ]:
import re

def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove non-alphanumeric characters
    text = text.lower()  # Convert text to lowercase
    text = ' '.join(text.split())  # Remove extra spaces
    return text


# Clean the text data and load the tokenizer
We apply the `clean_text` function to clean the 'text' column of the dataset.
Then, we initialize the PubMedBERT tokenizer to prepare for tokenization.


In [ ]:
# Load the tokenizer for the PubMedBERT model, specifically fine-tuned for biomedical text
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")

# Apply the text cleaning function to the 'text' column in the DataFrame
# This function will preprocess each text entry by removing unwanted characters, stopwords, etc.
df['text'] = df['text'].apply(clean_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Define features and targets for classification tasks
We specify the input features like date and country and set the classification targets.



In [ ]:
# Define the features for the model, which include the year, month, day, and country information
features = ['year', 'month', 'day', 'country']

# Define the target variables for Subtask 1, which are the hazard-category and product-category
targets_subtask1 = ['hazard-category', 'product-category']

# Define the target variables for Subtask 2, which are hazard and product
# Add other targets if necessary depending on the task
targets_subtask2 = ['hazard', 'product']


# Encode target labels
For classification, target labels need to be encoded as numeric values.
We use `LabelEncoder` to convert categorical labels into integers.


In [ ]:
# Create an empty dictionary to store label encoders for each target
label_encoders = {}

# Iterate over both sets of targets (Subtask 1 and Subtask 2)
for target in targets_subtask1 + targets_subtask2:
    # Initialize a LabelEncoder for each target
    le = LabelEncoder()

    # Transform the target column values into numeric labels and update the DataFrame
    df[target] = le.fit_transform(df[target])

    # Store the fitted LabelEncoder in the dictionary for future use (e.g., inverse transformation)
    label_encoders[target] = le


# Define a custom PyTorch dataset for text classification
This dataset class will handle text tokenization and label processing.
It ensures the text is properly encoded, padded, and truncated to a fixed length for the model.


In [ ]:
# Define a custom Dataset class for text data
class TextDataset(Dataset):
    # Initialize the dataset with texts, labels, tokenizer, and maximum sequence length
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts  # List of input texts
        self.labels = labels  # List of corresponding labels
        self.tokenizer = tokenizer  # Tokenizer for encoding the text
        self.max_len = max_len  # Maximum length for padding/truncation

    # Define the length of the dataset (number of samples)
    def __len__(self):
        return len(self.texts)

    # Define how to retrieve a single item from the dataset
    def __getitem__(self, item):
        text = str(self.texts[item])  # Get the text for the given index
        label = self.labels[item]  # Get the label for the given index

        # Use the tokenizer to encode the text (add special tokens, padding, truncation)
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # Add special tokens (e.g., [CLS], [SEP])
            max_length=self.max_len,  # Limit the sequence length
            padding='max_length',  # Pad sequences to max_length
            truncation=True,  # Truncate longer sequences
            return_tensors='pt'  # Return PyTorch tensors
        )

        # Return a dictionary with input_ids, attention_mask, and label
        return {
            'input_ids': encoding['input_ids'].flatten(),  # Flatten the tensor
            'attention_mask': encoding['attention_mask'].flatten(),  # Flatten the attention mask
            'label': torch.tensor(label, dtype=torch.long)  # Convert label to a tensor
        }


# Split the data into training and testing sets
We split the dataset into training and testing sets for each target.
This ensures that the model is trained on one set and evaluated on a separate, unseen set.


In [ ]:
# Define a function to prepare data for model training, validation, and testing
def prepare_data(text_column):
    # Extract features and text column from the DataFrame
    X = df[features + [text_column]]  # Features include specified columns plus the text column
    # Extract target variables for Subtask 1 and Subtask 2
    y_subtask1 = df[targets_subtask1]
    y_subtask2 = df[targets_subtask2]

    # Initialize a dictionary to store data splits for each target
    data_splits = {}

    # Iterate over both sets of target variables (Subtask 1 and Subtask 2)
    for target in targets_subtask1 + targets_subtask2:
        # Split the data into training (70%), validation (20%), and testing (10%) sets
        X_train, X_temp, y_train, y_temp = train_test_split(
            X, df[target], test_size=0.3, random_state=42
        )
        X_valid, X_test, y_valid, y_test = train_test_split(
            X_temp, y_temp, test_size=0.33, random_state=42  # This gives approximately 10% test and 20% validation
        )

        # Reset the indices for the train, validation, and test sets
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        X_valid = X_valid.reset_index(drop=True)
        y_valid = y_valid.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)

        # Store the splits for the current target in the dictionary
        data_splits[target] = (X_train, X_valid, X_test, y_train, y_valid, y_test)

    # Return the dictionary containing data splits for each target
    return data_splits

# Prepare the data splits for text-based tasks
We apply the `prepare_data` function specifically for text tasks and save the splits for later use.


In [ ]:
# Prepare the data splits for the 'text' column using the prepare_data function
text_splits = prepare_data('text')


# Set model configuration and define the device
Here, we configure key parameters for training like maximum sequence length, batch size, and learning rate.
We also determine whether to use GPU or CPU for training based on availability.


In [ ]:
# Define configuration settings for the model training
config = {
    'max_len': 512,  # Maximum sequence length for input texts
    'batch_size': 16,  # Batch size for training
    'learning_rate': 2e-5,  # Learning rate for the optimizer
    'epochs': 30,  # Increased number of training epochs
    'model_name': "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"  # Pre-trained model to use
}



# Determine the device to use for training (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


# Train and evaluate the model for each task
This function performs model training and evaluation for each target.
It uses a neural network to predict labels and calculates the F1 score for evaluation.


In [ ]:
# Train and evaluate the neural network for each target task
def train_and_evaluate_bert(data_splits, targets):
    # Initialize an empty list to store F1 scores for each target task
    f1_scores = []

    # Loop through each target (task) for training and evaluation
    for target in targets:
        print(f"\nStarting training for task: {target}")

        # Retrieve the corresponding training, validation, and testing splits
        X_train, X_valid, X_test, y_train, y_valid, y_test = data_splits[target]

        # Extract the 'text' column for training, validation, and testing
        texts_train = X_train['text'].values
        texts_valid = X_valid['text'].values
        texts_test = X_test['text'].values

        # Create datasets for training, validation, and testing
        train_dataset = TextDataset(texts_train, y_train, tokenizer, config['max_len'])
        valid_dataset = TextDataset(texts_valid, y_valid, tokenizer, config['max_len'])
        test_dataset = TextDataset(texts_test, y_test, tokenizer, config['max_len'])

        # Create data loaders for batching during training, validation, and testing
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

        # Determine the number of labels for classification dynamically based on the target task
        num_classes = len(label_encoders[target].classes_)  # This gives the number of unique classes for the current task

        # Calculate class weights for CrossEntropyLoss
        class_weights = compute_class_weight(
            'balanced',
            classes=np.unique(y_train),
            y=y_train
        )

        # Initialize the full class weights tensor with 1 (or any other default weight)
        class_weights_full = torch.ones(num_classes)  # Start with all weights equal to 1

        # Map the computed class weights to the appropriate indices for the classes that exist in the training set
        for i, class_idx in enumerate(np.unique(y_train)):
            class_weights_full[class_idx] = class_weights[i]

        # Move the class weights tensor to the correct device (GPU/CPU)
        class_weights_full = class_weights_full.to(device)

        # Load the pre-trained model with the appropriate number of labels for classification
        model = AutoModelForSequenceClassification.from_pretrained(config['model_name'], num_labels=num_classes).to(device)

        # Initialize the optimizer, loss function (with class weights), and learning rate scheduler
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True)
        criterion = nn.CrossEntropyLoss(weight=class_weights_full)

        # Early stopping setup
        best_val_loss = float('inf')
        best_epoch = 0
        patience = 10  # Number of epochs with no improvement before stopping
        epochs_without_improvement = 0
        best_model = None  # Variable to hold the best model

        # Training loop
        model.train()
        for epoch in range(config['epochs']):
            print(f"Epoch {epoch+1}/{config['epochs']} - Training: {target}")
            progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}", total=len(train_loader), leave=True)
            for batch in progress_bar:
                optimizer.zero_grad()

                # Get the input data and labels for the current batch
                input_ids = batch['input_ids'].squeeze(1).to(device)
                attention_mask = batch['attention_mask'].squeeze(1).to(device)
                labels = batch['label'].to(device)

                # Forward pass
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = criterion(outputs.logits, labels)

                # Backward pass and optimization step
                loss.backward()
                optimizer.step()

                # Update the progress bar with the current loss
                progress_bar.set_postfix(loss=loss.item())

            # Print the current learning rate
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Learning rate after epoch {epoch+1}: {current_lr}")

            # Validation step
            model.eval()
            val_loss = 0
            val_steps = 0
            val_preds = []
            val_true = []
            with torch.no_grad():
                for batch in valid_loader:
                    input_ids = batch['input_ids'].squeeze(1).to(device)
                    attention_mask = batch['attention_mask'].squeeze(1).to(device)
                    labels = batch['label'].to(device)

                    # Forward pass
                    outputs = model(input_ids, attention_mask=attention_mask)
                    loss = criterion(outputs.logits, labels)

                    val_loss += loss.item()
                    val_steps += 1

                    # Collect true labels and predictions for F1 score calculation
                    _, preds = torch.max(outputs.logits, dim=1)
                    val_preds.extend(preds.cpu().numpy())
                    val_true.extend(labels.cpu().numpy())

            # Calculate average validation loss
            avg_val_loss = val_loss / val_steps
            print(f"Validation Loss after Epoch {epoch+1}: {avg_val_loss}")

            # Calculate and print Macro F1 score for validation
            macro_f1 = f1_score(val_true, val_preds, average='macro')
            print(f"Validation Macro F1 Score after Epoch {epoch+1}: {macro_f1}")

            # Apply learning rate scheduler
            scheduler.step(avg_val_loss)

            # Save the model if it has the best validation loss so far
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_epoch = epoch + 1
                epochs_without_improvement = 0
                # Save the model and tokenizer
                best_model = model.state_dict()  # Save model weights
                print(f"New best model found. Saving the model at epoch {best_epoch}.")
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print("Early stopping triggered!")
                    break

        # After training, save the best model
        if best_model:
            print(f"Saving the best model from epoch {best_epoch}.")
            model.load_state_dict(best_model)  # Restore the best model
            model.save_pretrained(f'./best_model_{target}')
            tokenizer.save_pretrained(f'./best_model_{target}')
        else:
            print("No improvement in validation loss. No model saved.")

        # Evaluate the model on the test set
        print(f"Evaluating model for task: {target}")
        model.eval()
        y_preds = []
        y_true = []

        # Evaluate without computing gradients
        with torch.no_grad():
            for batch in tqdm(test_loader, desc="Evaluating", total=len(test_loader), leave=True):
                input_ids = batch['input_ids'].squeeze(1).to(device)
                attention_mask = batch['attention_mask'].squeeze(1).to(device)
                labels = batch['label'].to(device)

                # Forward pass
                outputs = model(input_ids, attention_mask=attention_mask)

                # Get predictions and append them to the list
                _, preds = torch.max(outputs.logits, dim=1)
                y_preds.extend(preds.cpu().numpy())
                y_true.extend(labels.cpu().numpy())

        # Decode the predictions and true labels
        decoded_preds = label_encoders[target].inverse_transform(y_preds)
        decoded_true = label_encoders[target].inverse_transform(y_true)

        # Calculate the macro F1 score
        f1 = f1_score(decoded_true, decoded_preds, average='macro')
        f1_scores.append(f1)
        print(f"F1-Score for {target}: {f1}")

        # Print the classification report
        print(f"Classification Report for {target}:\n")
        print(classification_report(decoded_true, decoded_preds, zero_division=0))

        # Save the label encoder for the current task
        np.save(f'./best_model_{target}/{target}_label_encoder.npy', label_encoders[target].classes_)
        print(f"Label Encoder for {target} saved in './best_model_{target}'")

    # Return the F1 scores for each target task
    return f1_scores

# Train and evaluate for all targets (subtasks 1 and 2)
text_f1_scores = train_and_evaluate_bert(text_splits, targets_subtask1 + targets_subtask2)


Starting training for task: hazard-category


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/30 - Training: hazard-category


Training Epoch 1: 100%|██████████| 223/223 [01:15<00:00,  2.96it/s, loss=0.463]


Learning rate after epoch 1: 2e-05
Validation Loss after Epoch 1: 0.9384824289008975
Validation Macro F1 Score after Epoch 1: 0.5587439761411614
New best model found. Saving the model at epoch 1.
Epoch 2/30 - Training: hazard-category


...

Learning rate after epoch 26: 2.5e-06
Validation Loss after Epoch 26: 6.874522358179092
Validation Macro F1 Score after Epoch 26: 0.027294120048236226
Epoch 27/30 - Training: product


Training Epoch 27: 100%|██████████| 223/223 [01:13<00:00,  3.04it/s, loss=1.87]


Learning rate after epoch 27: 1.25e-06
Validation Loss after Epoch 27: 6.870963163673878
Validation Macro F1 Score after Epoch 27: 0.027011889123490857
Early stopping triggered!
Saving the best model from epoch 17.
Evaluating model for task: product


Evaluating: 100%|██████████| 32/32 [00:04<00:00,  7.84it/s]


F1-Score for product: 0.03762012740408766
Classification Report for product:

                                                                        precision    recall  f1-score   support

                                                Catfishes (freshwater)       0.00      0.00      0.00         1
                                                 Fishes not identified       0.00      0.00      0.00         4
                                              Not classified pork meat       0.00      0.00      0.00         2
                                                 all purpose seasoning       1.00      1.00      1.00         1
                                                           almond milk       0.00      0.00      0.00         0
                                                         almond powder       0.00      0.00      0.00         1
                                                       almond products       0.00      0.00      0.00         2
                         

# Generate predictions on the test data and print the predictions DataFrame
Here, we load the test dataset, use the trained model to generate predictions, and display the results.


In [ ]:
# Import necessary libraries
import os
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder

# Load the test data for predictions (CSV containing validation data)
test_path = '/content/drive/MyDrive/Data/validation_data/incidents.csv'
test_df = pd.read_csv(test_path, index_col=0)

# Define the predict function
def predict(texts, model_base_path, target):
    # Load the tokenizer for the specified pre-trained model
    tokenizer = AutoTokenizer.from_pretrained(model_base_path)

    # Load the correct label encoder for the given target
    label_encoder_path = f'{model_base_path}/{target}_label_encoder.npy'
    label_encoder = LabelEncoder()

    # Check if the label encoder file exists and load it
    if os.path.exists(label_encoder_path):
        label_encoder.classes_ = np.load(label_encoder_path, allow_pickle=True)
    else:
        # Print a warning if the label encoder is not found
        print(f"Warning: Label encoder not found for {target} at {label_encoder_path}")
        return None

    # Load the pre-trained model for sequence classification
    model = AutoModelForSequenceClassification.from_pretrained(model_base_path).to(device)

    # Tokenize the input texts
    inputs = tokenizer(
        texts,
        padding=True,  # Pad sequences to the max length
        truncation=True,  # Truncate sequences to the max length
        max_length=512,  # Limit sequence length to 512 tokens
        return_tensors="pt"  # Return PyTorch tensors
    ).to(device)

    # Put the model in evaluation mode
    model.eval()

    # Make predictions with no gradient calculation
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)  # Get the predicted class for each input

    # Decode the predictions using the label encoder
    decoded_predictions = label_encoder.inverse_transform(predictions.cpu().numpy())

    # Return the decoded predictions
    return decoded_predictions

# Define device for model prediction (use GPU if available, else use CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Prepare an empty dataframe to store the predictions
predictions = pd.DataFrame()

# Run predictions for all targets using the correct saved model
for column in targets_subtask1 + targets_subtask2:
    # Define the model path dynamically based on the target column
    model_path = f'./best_model_{column}'  # Update model path to point to the best model

    # Get the decoded predictions for the current target
    decoded_preds = predict(test_df['text'].tolist(), model_path, column)

    # If predictions were successfully made, store them in the dataframe
    if decoded_preds is not None:
        predictions[column] = decoded_preds

# Display the final predictions
print("\nFinal Predictions:\n")
print(predictions)


Using device: cuda

Final Predictions:

    hazard-category                                   product-category  \
0        biological                       meat, egg and dairy products   
1        biological                       meat, egg and dairy products   
2        biological                       meat, egg and dairy products   
3         allergens                                  ices and desserts   
4    foreign bodies                       meat, egg and dairy products   
..              ...                                                ...   
560       allergens                              fruits and vegetables   
561       allergens  dietetic foods, food supplements, fortified foods   
562  foreign bodies                        cereals and bakery products   
563       allergens                        cereals and bakery products   
564       allergens       cocoa and cocoa preparations, coffee and tea   

                           hazard          product  
0                 

In [ ]:
predictions

,hazard-category,product-category,hazard,product
0,biological,"meat, egg and dairy products",listeria spp,milk
1,biological,"meat, egg and dairy products",escherichia coli,chicken breast
2,biological,"meat, egg and dairy products",enteroviruses,chicken breast
3,allergens,ices and desserts,pecan nut,dried apricots
4,foreign bodies,"meat, egg and dairy products",plastic fragment,dried beef meat
...,...,...,...,...
560,allergens,fruits and vegetables,cashew,fruit pies
561,allergens,"dietetic foods, food supplements, fortified foods",milk and products thereof,lettuce
562,foreign bodies,cereals and bakery products,plastic fragment,mushrooms
563,allergens,cereals and bakery products,peanuts and products thereof,flour


# Create the submission folder and archive the results
Finally, predictions and models are saved into a submission directory for easy sharing or evaluation.


In [ ]:
import os
from shutil import make_archive
import pandas as pd
from google.colab import drive

# Define the Google Drive path where you want to save the files
output_folder = '/content/drive/MyDrive/submission_finetunedPUBMEDBERTv2/'

# Create the folder in Google Drive if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Save predictions to a CSV file named 'submission.csv' inside the folder
predictions.to_csv(f'{output_folder}submission.csv', index=False)

# Zip the folder for submission
make_archive(output_folder, 'zip', output_folder)

# Print confirmation message
print(f"Submission saved to Google Drive at {output_folder}")


Submission saved to Google Drive at /content/drive/MyDrive/submission_finetunedPUBMEDBERTv2/
